# Tarea Dask

## Instrucciones generales
- Esta tarea debe realizarse de manera individual
- Este notebook (resuelto) debe ser subido al github del proyecto en la carpeta de tareas (creen una carpeta dentro de esa carpeta y agreguen su notebook reuelto)
- Fecha límite: Lunes 25 de noviembre de 2024 a las 11:59 p.m
- Deben realizar las cuatro secciones
- Puedes agregar tantas celdas de código y explicaciones como veas necesario, solo manten la estructura general

## Sección 0 Creación y Configuración del cliente de Dask
Ejercicio 0: Configuración del cliente
1. Crea un cliente local de Dask que inicie un clúster en tu máquina.
2. Configura el cliente para que tenga las siguientes características (elige un par de las opciones de trabajadores e hilos):
    - Número de trabajadores: 2 / 4
    - Memoria máxima por trabajador: 1GB
    - Threads por trabajador: 4 / 2
3. Verifica que el cliente esté funcionando correctamente mostrando:
    - Resumen de los trabajadores activos.
    - Dashboard disponible (URL del panel de control de Dask).
    * Tip: Checa los parámetros del cliente que creeaste.

*Nota*: Puedes hacer que corra en el puerto que desees.

In [ ]:
# Tu código va aquí
from dask.distributed import Client

client = Client(
    n_workers = 2, 
    threads_per_worker=4,
    memory_limit='1GB',
)

ModuleNotFoundError: No module named 'dask'

## Sección 1 Delayed
Ejercicio 1: Procesamiento de datos 

1. Genera datos simulados (por ejemplo, ventas diarias) para 10 sucursales durante 365 días.
    - Cada sucursal debe tener datos generados aleatoriamente para "Ingresos" y "Costos".
    - Utiliza una función para generar los datos simulados.
2. Usa Dask Delayed para calcular:
    - Las ganancias diarias por sucursal.
    - La sucursal con mayor ganancia promedio.
3. Genera un grafo de tareas que visualice estas operaciones y explica por qué elegiste paralelizar de esa forma, genera una visualización del grafo.

In [ ]:
import pandas as pd
import numpy as np
from dask import delayed
from dask import visualize
# Paso 1: Generar datos simulados
def generar_datos_sucursal(sucursal_id):
    fechas = pd.date_range(start='2022-01-01', periods=365, freq='D')
    ingresos = np.random.uniform(1000, 5000, size=365)
    costos = np.random.uniform(500, 3000, size=365)
    datos = pd.DataFrame({
        'Fecha': fechas,
        'Sucursal': sucursal_id,
        'Ingresos': ingresos,
        'Costos': costos
    })
    return datos

# Generar datos para 10 sucursales
sucursales = []
for i in range(1, 11):
    sucursal_datos = generar_datos_sucursal(f'Sucursal_{i}')
    sucursales.append(sucursal_datos)

# Concatenar datos de todas las sucursales
datos_totales = pd.concat(sucursales, ignore_index=True)
print(datos_totales)


# Paso 2: Calcular ganancias diarias por sucursal usando Dask Delayed
@delayed
def calcular_ganancia_diaria(datos_sucursal):
    datos_sucursal['Ganancia'] = datos_sucursal['Ingresos'] - datos_sucursal['Costos']
    return datos_sucursal

# Aplicar la función retrasada a cada sucursal
tareas_ganancia = []
for sucursal_id in datos_totales['Sucursal'].unique():
    datos_sucursal = datos_totales[datos_totales['Sucursal'] == sucursal_id]
    tarea = calcular_ganancia_diaria(datos_sucursal)
    tareas_ganancia.append(tarea)

# Paso 3: Calcular la ganancia promedio por sucursal
@delayed
def calcular_ganancia_promedio(datos_sucursal):
    ganancia_promedio = datos_sucursal['Ganancia'].mean()
    return (datos_sucursal['Sucursal'].iloc[0], ganancia_promedio)

# Aplicar la función retrasada para calcular la ganancia promedio
tareas_ganancia_promedio = [calcular_ganancia_promedio(tarea) for tarea in tareas_ganancia]

# Encontrar la sucursal con mayor ganancia promedio
@delayed
def sucursal_mayor_ganancia(lista_ganancias):
    return max(lista_ganancias, key=lambda x: x[1])

tarea_mayor_ganancia = sucursal_mayor_ganancia(tareas_ganancia_promedio)

# Paso 4: Generar y visualizar el grafo de tareas
# Visualizar el grafo de tareas (el archivo se guardará como 'grafo_tareas.png')
tarea_mayor_ganancia.visualize()
# Paso 5: Ejecutar las tareas y obtener el resultado
resultado = tarea_mayor_ganancia.compute()
print(f"La sucursal con mayor ganancia promedio es {resultado[0]} con una ganancia promedio de {resultado[1]:.2f}")

## Sección 2 Dask Dataframes
Ejercicio 2: Limpieza y análisis de datos reales

1. Descarga un conjunto de datos masivo (puedes usar la colección de *nycflights* que se encuentra en `data/nycflights/`).
2. Carga los datos en un Dask DataFrame. 
    - Elige adecuadamente el número de particiones (que quepan en memoria de los `workers`)
3. Realiza las siguientes tareas:
    - Limpia los valores faltantes en las columnas `ArrDelay` y `DepDelay`, rellenándolos con la mediana de cada columna.
    - Calcula el retraso promedio (`DepDelay`) por mes y aerolínea.
    - Encuentra el aeropuerto de origen con más vuelos retrasados.

*Nota*: **Evita** convertir el DataFrame a pandas e **intenta** realizar `.compute()` solo cuando sea necesario.

In [ ]:
# Tu código va aquí
import dask.dataframe as dd

# Cargar el archivo CSV en un Dask DataFrame
# Ajusta la ruta si es necesario según la ubicación de los datos
ruta_archivo = 'data/nycflights/*.csv'  # Ruta que contiene múltiples archivos CSV de nycflights
df = dd.read_csv(ruta_archivo, dtype = dtype)

# Dividir en un número adecuado de particiones (dependerá de la memoria disponible)
# En este caso, se elige dividir en 10 particiones, pero puedes ajustar según tu máquina
df = df.repartition(npartitions=10)

mediana_ArrDelay = df["ArrDelay"].median_approximate()
if(df["ArrDelay"].isna):
    df["ArrDelay"].apply(
    lambda x: mediana_ArrDelay, meta=('x','float')
    )
mediana_DepDelay = df["DepDelay"].median_approximate()
if(df["DepDelay"].isna):
    df["DepDelay"].apply(
    lambda x: mediana_DepDelay, meta=('x','float')
    )
delay_by_month_airline = df.groupby(['Month', 'UniqueCarrier'])['DepDelay'].mean()
delayed_flights = df[df['DepDelay'] > 0]
origin_with_most_delays = delayed_flights['Origin'].value_counts().idxmax().compute()

print("Retraso promedio por mes y aerolínea:")
print(delay_by_month_airline.compute())

print("\nAeropuerto de origen con más vuelos retrasados:")
print(origin_with_most_delays)


## Sección 3 Dask Arrays

Ejercicio 3: Procesamiento numérico avanzado

1. Crea un arreglo de 10,000 x 10,000 con valores aleatorios usando Dask Array, utiliza un tamaño de chunks adecuado, ¿es mejor que sean cuadrados?.
2. Realiza las siguientes operaciones:
    - Calcula la suma de cada fila.
    - Encuentra la fila con el valor máximo promedio.
    - Multiplica todo el arreglo por un factor escalar (por ejemplo, 2.5).
3. Divide el arreglo nuevamente en 100 bloques y compara la rapidez.

In [ ]:
# Tu código va aquí
import dask.array as da
import time

# Definir el tamaño del arreglo
shape = (10000, 10000)

# Crear arreglos Dask con diferentes tamaños de chunks
arr_cuadrado = da.random.random(size=shape, chunks=(1000, 1000))
arr_filas = da.random.random(size=shape, chunks=(10000, 1000))
arr_columnas = da.random.random(size=shape, chunks=(1000, 10000))

def realizar_operaciones(arr):
    inicio = time.time()
    
    # Calcular la suma de cada fila
    sumas_filas = arr.sum(axis=1)
    sumas_filas_resultado = sumas_filas.compute()
    
    # Encontrar la fila con el valor máximo promedio
    promedios_filas = arr.mean(axis=1)
    indice_max_promedio = promedios_filas.argmax()
    indice_max_promedio_resultado = indice_max_promedio.compute()
    
    # Multiplicar todo el arreglo por un factor escalar (2.5)
    arr_escalado = arr * 2.5
    arr_escalado_resultado = arr_escalado.compute()
    
    fin = time.time()
    tiempo_total = fin - inicio
    
    return tiempo_total, indice_max_promedio_resultado

# Operaciones con chunks cuadrados
tiempo_cuadrado, indice_max_cuadrado = realizar_operaciones(arr_cuadrado)
print(f"Tiempo con chunks cuadrados (1000, 1000): {tiempo_cuadrado:.2f} segundos")
print(f"Fila con el promedio máximo: {indice_max_cuadrado}")

# Operaciones con chunks grandes en filas
tiempo_filas, indice_max_filas = realizar_operaciones(arr_filas)
print(f"Tiempo con chunks en filas (10000, 1000): {tiempo_filas:.2f} segundos")
print(f"Fila con el promedio máximo: {indice_max_filas}")

# Operaciones con chunks grandes en columnas
tiempo_columnas, indice_max_columnas = realizar_operaciones(arr_columnas)
print(f"Tiempo con chunks en columnas (1000, 10000): {tiempo_columnas:.2f} segundos")
print(f"Fila con el promedio máximo: {indice_max_columnas}")

# Rechunking a 100 bloques y comparación
arr_rechunk = arr_cuadrado.rechunk((1000, 1000))
tiempo_rechunk, indice_max_rechunk = realizar_operaciones(arr_rechunk)
print(f"Tiempo después de rechunking (1000, 1000): {tiempo_rechunk:.2f} segundos")
print(f"Fila con el promedio máximo: {indice_max_rechunk}")

## Sección 4 Futures
Ejercicio 4: Distribución de tareas dinámicas

1. Implementa una función que calcule la raíz cuadrada de una lista de 100,000 números enteros generados aleatoriamente.
2. Divide la lista en 10 partes iguales y usa Dask Futures para calcular la raíz cuadrada de cada parte en paralelo.
3. Recolecta los resultados y calcula:
    - El promedio de todos los números procesados.
    - El tiempo total de ejecución (incluyendo envío y recolección de tareas).
4. Observa como se distribuye la carga en el cliente.

*Nota*: en los ejercicios ya vimos como determinar si ya se cumplío una tarea.

In [ ]:
# Tu código va aquí
# Tu código va aquí
import dask
from dask.distributed import Client
import numpy as np
import time

# Inicializar el cliente Dask
client = Client()

def compute_sqrt(numbers):
    """Función que calcula la raíz cuadrada de una lista de números."""
    return np.sqrt(numbers)

# Generar la lista de 100,000 números enteros aleatorios
np.random.seed(42)  # Para reproducibilidad
numbers = np.random.randint(1, 1000000, size=100000)

# Dividir la lista en 10 partes iguales
partitions = np.array_split(numbers, 10)

# Medir el tiempo total de ejecución
start_time = time.time()

# Enviar tareas al cluster usando Dask Futures
futures = [client.submit(compute_sqrt, part) for part in partitions]

# Recolectar los resultados
results = client.gather(futures)

# Calcular el promedio de todos los números procesados
all_results = np.concatenate(results)
average_result = np.mean(all_results)

# Tiempo total de ejecución
total_time = time.time() - start_time

# Mostrar resultados
print(f"Promedio de los números procesados: {average_result}")
print(f"Tiempo total de ejecución: {total_time:.2f} segundos")

# Mostrar el cliente Dask
print("\nDistribución de carga:")
print(client)

# Cerrar el cliente
client.close()